In [ ]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from scipy.linalg import sqrtm
from torchvision.models import inception_v3
import torchvision.transforms.functional as TF
from PIL import Image
import pandas as pd
import os
import multiprocessing

In [ ]:
# Function to compute Fréchet Inception Distance (FID)
def calculate_fid(real_images, generated_images, device='mps'):
    real_images = real_images.detach().cpu().numpy()
    generated_images = generated_images.detach().cpu().numpy()

    # Preprocess images
    real_images = np.transpose(real_images, (0, 2, 3, 1))  # Change from NCHW to NHWC
    generated_images = np.transpose(generated_images, (0, 2, 3, 1))

    # Load pre-trained Inception v3 model
    inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
    inception_model.eval()

    # Compute activations for real and generated images
    with torch.no_grad():
        real_activations = inception_model(torch.tensor(real_images).to(device)).detach().cpu().numpy()
        generated_activations = inception_model(torch.tensor(generated_images).to(device)).detach().cpu().numpy()

    # Compute mean and covariance of activations
    mu_real = np.mean(real_activations, axis=0)
    mu_generated = np.mean(generated_activations, axis=0)
    cov_real = np.cov(real_activations, rowvar=False)
    cov_generated = np.cov(generated_activations, rowvar=False)

    # Compute FID
    fid = np.sum((mu_real - mu_generated) ** 2) + np.trace(cov_real + cov_generated - 2 * sqrtm(np.dot(cov_real, cov_generated)))
    return fid

In [ ]:
with torch.no_grad():
    # Ensure images are moved to CPU and converted for metric calculation
    # right now we are only working with single images
    real_image_np = real_A.cpu().numpy().transpose(0, 2, 3, 1)
    fake_image_np = fake_B.cpu().numpy().transpose(0, 2, 3, 1) 
    real_image_np = np.squeeze(real_image_np, axis = 0) # get rid of first channel because it is just batch size 1
    fake_image_np = np.squeeze(fake_image_np, axis = 0)
                # Clipping pixel values to ensure they are within [0, 1]
    print("Real image: ", real_image_np)
    print("Fake image: ", fake_image_np)
    real_image_np = np.clip(real_image_np, 0, 1)
    fake_image_np = np.clip(fake_image_np, 0, 1)

    print(real_image_np.shape, fake_image_np.shape)
    # print("Real image: ", real_image_np)
    # print("Fake image: ", fake_image_np)

    image_ssim = ssim(real_image_np, fake_image_np, multichannel=True, channel_axis = 2, dtype = np.float32, data_range =[0,1])
    image_psnr = psnr(real_image_np, fake_image_np)

    # Append scores
    ssim_scores.append(image_ssim)
    psnr_scores.append(image_psnr)